Credit where credit is due: this is based on  https://www.kaggle.com/code/gpreda/fast-test-of-llama-v2-pre-quantized-with-llama-cpp.

Loading the model will take a while so please be patient

In [1]:
# clean up after failed efforts if necessary
!rm -rf llama.cpp
# we need no CUDA for CPU; this flag has changed since the initial release
!CMAKE_ARGS="-DLLAMA_CUDA=off" pip install llama-cpp-python 
!git clone https://github.com/ggerganov/llama.cpp.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 MB 19.2 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... - \ | / - \ | done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.78-cp310-cp310-linux_x86_64.whl size=3658479 sha256=a1d0a4fb40265d45e9d32ed21e972e9664a5b6af39173c77fce8872de714915d
  Stored in directory: /root/.cache/pip/wheels/fd/c5/bd/3b1c20081bd71ce9d28b562573c97915c790bf1ef231879a61
Successfully built llama-cpp-python
Cloning into 'llama.cpp'...
remote: Enumerating objects: 27254, done.
remote: Counting objects: 100% (5058/5058), done.
remote: Compressing objects: 100% (409/409), done.
remote: Total 27254 (delta 4787), reused 4785 (delta 4647), pack-reused 22196
Receiving objects: 100% (27254/2

In [2]:
import arrow
from llama_cpp import Llama

time_load = arrow.now()
llm = Llama(model_path="/kaggle/input/test-llama-2-quantized-with-llama-cpp/llama-7b.gguf")
print("Model load time: {:5.4f} seconds".format((arrow.now() - time_load).total_seconds()))

llama_model_loader: loaded meta data with 18 key-value pairs and 291 tensors from /kaggle/input/test-llama-2-quantized-with-llama-cpp/llama-7b.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = 7b-chat-hf
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.

Model load time: 53.3105 seconds


AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 1 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | 
Model metadata: {'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '2', 'general.architecture': 'llama', 'llama.context_length': '2048', 'general.name': '7b-chat-hf', 'llama.embedding_length': '4096', 'llama.feed_forward_length': '11008', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'llama.rope.dimension_count': '128', 'llama.attention.head_count': '32', 'tokenizer.ggml.bos_token_id': '1', 'llama.block_count': '32', 'llama.attention.head_count_kv': '32', 'tokenizer.ggml.model': 'llama', 'general.file_type': '7'}
Using fallback chat format: llama-2


In [3]:
def ask(query: str, max_tokens: int):
    time_ask = arrow.now()
    output = llm("Q: {} A: ".format(query), max_tokens=max_tokens, stop=["Q:", "\n"], echo=False)
    print(output)
    return [output['choices'][index]['text'] for index in range(len(output['choices']))], (arrow.now() - time_ask).total_seconds()

query_cities = 'What are the names of three European capital cities?'
result_cities, time_cities = ask(query=query_cities, max_tokens=40)
print(query_cities)
print(result_cities)
print('time: {:5.4f}'.format(time_cities))


llama_print_timings:        load time =    3308.80 ms
llama_print_timings:      sample time =      12.63 ms /    19 runs   (    0.66 ms per token,  1504.83 tokens per second)
llama_print_timings: prompt eval time =    3308.61 ms /    16 tokens (  206.79 ms per token,     4.84 tokens per second)
llama_print_timings:        eval time =   16494.68 ms /    18 runs   (  916.37 ms per token,     1.09 tokens per second)
llama_print_timings:       total time =   19844.46 ms /    34 tokens


{'id': 'cmpl-9bd0f212-c487-4e74-9286-87378bcee5f6', 'object': 'text_completion', 'created': 1718483704, 'model': '/kaggle/input/test-llama-2-quantized-with-llama-cpp/llama-7b.gguf', 'choices': [{'text': '1. Berlin, Germany 2. Paris, France 3. London, United Kingdom', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 16, 'completion_tokens': 18, 'total_tokens': 34}}
What are the names of three European capital cities?
['1. Berlin, Germany 2. Paris, France 3. London, United Kingdom']
time: 19.8575


In [4]:
query_catcher = 'What can you tell me about the J. D. Salinger novel The Catcher in the Rye?'
result_catcher, time_catcher = ask(query=query_catcher, max_tokens=100)
print(query_catcher)
print(result_catcher)
print('time: {:5.4f}'.format(time_catcher))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3308.80 ms
llama_print_timings:      sample time =      39.97 ms /    65 runs   (    0.61 ms per token,  1626.38 tokens per second)
llama_print_timings: prompt eval time =    4399.71 ms /    25 tokens (  175.99 ms per token,     5.68 tokens per second)
llama_print_timings:        eval time =   35909.73 ms /    64 runs   (  561.09 ms per token,     1.78 tokens per second)
llama_print_timings:       total time =   40427.93 ms /    89 tokens


{'id': 'cmpl-dc144f85-3597-483d-a0a6-aed8231eae70', 'object': 'text_completion', 'created': 1718483723, 'model': '/kaggle/input/test-llama-2-quantized-with-llama-cpp/llama-7b.gguf', 'choices': [{'text': ' The Catcher in the Rye, written by J.D. Salinger in 1951, is a classic coming-of-age story that has become one of the most widely read and influential novels of the 20th century. Here are some key points about the book:', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 29, 'completion_tokens': 65, 'total_tokens': 94}}
What can you tell me about the J. D. Salinger novel The Catcher in the Rye?
[' The Catcher in the Rye, written by J.D. Salinger in 1951, is a classic coming-of-age story that has become one of the most widely read and influential novels of the 20th century. Here are some key points about the book:']
time: 40.4451
